In [ ]:
# This code is prepared by Orhan Erdem
# Please email orhanerdem at gmail.com for errors, suggestions

In [2]:
from pathlib import Path

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
import statsmodels.formula.api as sm
import matplotlib.pylab as plt
from sklearn import metrics

import dmba
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

%matplotlib inline

In [3]:
#Load data
asses_df=pd.read_excel('2023_Roxbury.xlsx')
asses_df=asses_df.dropna()
asses_df.head()

,LU,LAND_SF,GROSS_AREA,TOTAL_VALUE,GROSS_TAX,YR_BUILT,YR_REMODEL,BED_RMS,FULL_BTH,HLF_BTH,KITCHENS,TT_RMS,HEAT_TYPE,FIREPLACES,NUM_PARKING,PROP_VIEW
2,R2,2588.0,3630.0,598700,6430.0400,1890.0,1,9.0,2.0,0.0,2.0,13.0,F - Forced Hot Air,0.0,2.0,A - Average
4,R3,2875.0,6276.0,848100,9108.5972,1890.0,0,9.0,3.0,0.0,3.0,18.0,W - Ht Water/Steam,0.0,3.0,A - Average
5,R3,3884.0,6411.0,803500,8629.5872,1905.0,0,9.0,3.0,0.0,3.0,18.0,W - Ht Water/Steam,0.0,4.0,A - Average
6,R3,3502.0,6083.0,1006800,10813.0300,1920.0,0,4.0,2.0,1.0,1.0,6.0,W - Ht Water/Steam,0.0,0.0,A - Average
7,R3,3502.0,6083.0,1006800,10813.0300,1900.0,1,6.0,3.0,0.0,3.0,12.0,F - Forced Hot Air,0.0,5.0,F - Fair


In [4]:
asses_df.describe()

,LAND_SF,GROSS_AREA,TOTAL_VALUE,GROSS_TAX,YR_BUILT,YR_REMODEL,BED_RMS,FULL_BTH,HLF_BTH,KITCHENS,TT_RMS,FIREPLACES,NUM_PARKING
count,3582.000000,3582.000000,3.582000e+03,3582.000000,3582.000000,3582.000000,3582.000000,3582.000000,3582.000000,3582.000000,3582.000000,3582.000000,3582.000000
mean,2836.093802,3110.809883,5.815996e+05,6166.272268,1929.290341,0.630095,4.240927,2.042434,0.352596,1.648520,8.810162,0.178671,1.255444
std,2255.182273,1824.103881,2.460601e+05,2704.682403,49.547430,0.730107,2.262270,1.007588,0.555662,0.821807,4.225523,0.596380,1.626133
min,440.000000,440.000000,1.176270e+05,0.000000,1800.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.000000,0.000000
25%,1354.000000,1445.250000,4.026000e+05,4308.347500,1890.000000,0.000000,3.000000,1.000000,0.000000,1.000000,5.000000,0.000000,0.000000
50%,2226.500000,2949.500000,5.522500e+05,5910.222200,1905.000000,0.000000,4.000000,2.000000,0.000000,1.000000,8.000000,0.000000,1.000000
75%,3721.500000,4355.500000,7.258000e+05,7763.944700,1989.000000,1.000000,6.000000,3.000000,1.000000,2.000000,12.000000,0.000000,2.000000
max,34273.000000,10685.000000,2.374700e+06,25504.280000,2022.000000,2.000000,14.000000,9.000000,3.000000,4.000000,20.000000,8.000000,16.000000


In [5]:
#create a list of predictor variables
#create a list of predicto variables
excludeColumns=('TOTAL_VALUE','LU','HEAT_TYPE','PROP_VIEW')
predictors=[s for s in asses_df.columns if s not in excludeColumns]
outcome='TOTAL_VALUE'

X=asses_df[predictors]
y=asses_df[outcome]

train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.4, random_state=1)

# train linear regression model
reg = LinearRegression()
reg.fit(train_X, train_y)

LinearRegression()

In [6]:
print('intercept',reg.intercept_)
print(pd.DataFrame({'Predictor':X.columns,'coefficient':reg.coef_}))

#Prediction
y_valid_pred=reg.predict(valid_X)
y_train_pred=reg.predict(train_X)

intercept -206765.2153103334
      Predictor   coefficient
0       LAND_SF     -0.936722
1    GROSS_AREA     10.700420
2     GROSS_TAX     76.377630
3      YR_BUILT    117.435263
4    YR_REMODEL  10124.635936
5       BED_RMS  -1577.506625
6      FULL_BTH  15538.172898
7       HLF_BTH  13890.034373
8      KITCHENS     94.449514
9        TT_RMS   2656.387370
10   FIREPLACES   3783.310203
11  NUM_PARKING      9.966388


# Training Part Evaluation

In [10]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(train_y, y_train_pred)
rmse=mean_squared_error(train_y, y_train_pred,squared=False)
print("Mean Squared Error:", mse.round(2))
print("Root Mean Squared Error:", rmse.round(2))

Mean Squared Error: 6100468727.14
Root Mean Squared Error: 78105.5


C:\Users\babebe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [9]:
regressionSummary(train_y,reg.predict(train_X))


Regression statistics

                      Mean Error (ME) : 0.0000
       Root Mean Squared Error (RMSE) : 78105.4974
            Mean Absolute Error (MAE) : 27861.2601
          Mean Percentage Error (MPE) : -1.6172
Mean Absolute Percentage Error (MAPE) : 5.7576


# Validation Part Evaluation

In [11]:
from sklearn.metrics import mean_squared_error
mse=mean_squared_error(valid_y, y_valid_pred)
rmse=mean_squared_error(valid_y, y_valid_pred,squared=False)
print("Mean Squared Error:", mse.round(2))
print("Root Mean Squared Error:", rmse.round(2))

Mean Squared Error: 3583622101.87
Root Mean Squared Error: 59863.36


C:\Users\babebe\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [12]:
regressionSummary(valid_y,reg.predict(valid_X))


Regression statistics

                      Mean Error (ME) : -918.7462
       Root Mean Squared Error (RMSE) : 59863.3619
            Mean Absolute Error (MAE) : 28309.8071
          Mean Percentage Error (MPE) : -1.4364
Mean Absolute Percentage Error (MAPE) : 6.1946


# R2 Akaike Information Criterion (AIC) and Bayesian Information Criterion(BIC)

In [13]:
#Calcualte R2
from sklearn.metrics import r2_score

r2=r2_score(train_y, y_train_pred)
p=len(predictors)
n=len(train_y)
adjusted_r2=1-(1-r2)*(n-1)/(n-p-1)

print("R2:",r2.round(2))
print("Adjusted R2", adjusted_r2.round(2))

R2: 0.89
Adjusted R2 0.89


In [14]:
print('adjusted r2',adjusted_r2_score(train_y,y_train_pred,reg).round(2))
print('AIC',AIC_score(train_y, y_train_pred,reg).round(2))
print('BIC',BIC_score(train_y, y_train_pred,reg).round(2))

adjusted r2 0.89
AIC 54547.07
BIC 54626.49


In [15]:
#Use predict() to make prediction on a new set (validation set, for example)
reg_pred=reg.predict(valid_X)

result=pd.DataFrame({'Predicted':reg_pred,'Actual': valid_y,
                    'Residual':valid_y-reg_pred})

print(result.head(20))

#Compute also common accuracy metrics
regressionSummary(valid_y,reg_pred)

          Predicted  Actual      Residual
805   869505.942030  929700  60194.057970
5766  457763.019923  457600   -163.019923
189   405455.534288  393100 -12355.534288
6038  362853.182817  335400 -27453.182817
4582  680656.337252  671300  -9356.337252
1157  489275.379171  495000   5724.620829
4910  611823.974828  621000   9176.025172
5036  609971.743041  602800  -7171.743041
4049  774753.755597  772200  -2553.755597
4683  682267.191486  661900 -20367.191486
2740  412775.379384  414600   1824.620616
1820  849590.750881  856700   7109.249119
2248  890752.613598  879600 -11152.613598
4093  696737.727366  723100  26362.272634
5621  438688.443518  409000 -29688.443518
435   670676.695975  692900  22223.304025
5537  476482.467320  394700 -81782.467320
5238  479191.173293  472600  -6591.173293
5434  343064.771666  341600  -1464.771666
1736  485833.148052  446200 -39633.148052

Regression statistics

                      Mean Error (ME) : -918.7462
       Root Mean Squared Error (RMSE) : 5986

# Backward Elimination

In [16]:
def train_model(variables):
    model=LinearRegression()
    model.fit(train_X[variables],train_y)
    return model

def score_model (model, variables):
    return AIC_score (train_y, model.predict(train_X[variables]),model)

best_model, best_variables=backward_elimination(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

Variables: LAND_SF, GROSS_AREA, GROSS_TAX, YR_BUILT, YR_REMODEL, BED_RMS, FULL_BTH, HLF_BTH, KITCHENS, TT_RMS, FIREPLACES, NUM_PARKING
Start: score=54547.07
Step: score=54545.07, remove NUM_PARKING
Step: score=54543.07, remove KITCHENS
Step: score=54541.78, remove LAND_SF
Step: score=54540.44, remove BED_RMS
Step: score=54540.28, remove FIREPLACES
Step: score=54540.28, remove None
['GROSS_AREA', 'GROSS_TAX', 'YR_BUILT', 'YR_REMODEL', 'FULL_BTH', 'HLF_BTH', 'TT_RMS']


In [17]:
regressionSummary(valid_y, best_model.predict(valid_X[best_variables]))


Regression statistics

                      Mean Error (ME) : -1042.0621
       Root Mean Squared Error (RMSE) : 59738.6808
            Mean Absolute Error (MAE) : 28177.1382
          Mean Percentage Error (MPE) : -1.4576
Mean Absolute Percentage Error (MAPE) : 6.1831


# Forward Selection Model

In [18]:
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

Variables: LAND_SF, GROSS_AREA, GROSS_TAX, YR_BUILT, YR_REMODEL, BED_RMS, FULL_BTH, HLF_BTH, KITCHENS, TT_RMS, FIREPLACES, NUM_PARKING
Start: score=59333.85, constant
Step: score=54769.35, add GROSS_TAX
Step: score=54620.53, add GROSS_AREA
Step: score=54576.58, add FULL_BTH
Step: score=54555.90, add HLF_BTH
Step: score=54546.26, add YR_REMODEL
Step: score=54543.74, add YR_BUILT
Step: score=54540.28, add TT_RMS
Step: score=54540.28, add None
['GROSS_TAX', 'GROSS_AREA', 'FULL_BTH', 'HLF_BTH', 'YR_REMODEL', 'YR_BUILT', 'TT_RMS']


# Regression Output

In [19]:
train_df=train_X.join(train_y)

predictors= train_X.columns
formula='TOTAL_VALUE~'+'+'.join(predictors)

reg=sm.ols(formula=formula,data=train_df).fit()
print(reg.summary())

                            OLS Regression Results                            
Dep. Variable:            TOTAL_VALUE   R-squared:                       0.893
Model:                            OLS   Adj. R-squared:                  0.893
Method:                 Least Squares   F-statistic:                     1492.
Date:                Fri, 05 Apr 2024   Prob (F-statistic):               0.00
Time:                        16:28:14   Log-Likelihood:                -27260.
No. Observations:                2149   AIC:                         5.455e+04
Df Residuals:                    2136   BIC:                         5.462e+04
Df Model:                          12                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept   -2.068e+05   8.73e+04     -2.369      